In [ ]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

os.chdir('../../')

In [ ]:
df = pd.read_csv('data/processed/preprocessed.csv')

### Rule метод через множество соединённых классов

Пока разметка классов идёт по первым встреченным словам с помощью просто нахождения слов.  
Проблема в том, что одна строка может иметь много классов, что переходит в многолейбловую классификацию, что требует намного больший датасет.  
Также среди размеченных данных могут быть такие классы, которые встречаются только в числе других классов, а не одиночно.  
В итоге метод не очень подходит под этот датасет, но можно посмотреть классы.

In [ ]:
import re 

# Функции чтобы искать именно слова, а не подстроки
# Хотя бы одно слово
def has_any_words(words, text):
    if not words:
        return False
    pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    return bool(re.search(pattern, text, flags=re.IGNORECASE))

# Хотя бы все слова
def has_all_words(words, text):
    for word in words:
        pattern = r'\b' + re.escape(word) + r'\b'
        if not re.search(pattern, text, flags=re.IGNORECASE):
            return False
    return True

# функция не обновлялась по сравнению с обычным rule методом, так как неудачно, но осталось истории ради
def extract_status(text):
    statuses = []

    if has_all_words(['подлежащее удовлетворение'], text):
        statuses.append('Удовлетворено')
    if has_any_words(['не_подлежащее', 'не_удовлетворение'], text):
        if has_all_words(['подлежащее', 'удовлетворение'], text):
            statuses.append('Частично удовлетворено')
        statuses.append('Отказ')
    if has_all_words(['ип', 'приостановлено'], text):
        statuses.append('ИП приостановлено')
    if has_any_words(['приостановить'], text):
        statuses.append('Приостановлено')
    if has_all_words(['окончить', 'производство'], text):
        statuses.append('Производство окончено')
    if has_all_words(['обратить', 'взыскание'], text) or has_all_words(['обращение', 'взыскание'], text):
        statuses.append('Взыскание обращено')
    if has_any_words(['действие', 'рег', 'регистрационный'], text) and 'запрет' in text:
        statuses.append('Запрет действий')
    if has_all_words(['невозможно', 'установить'], text) or has_any_words(['не_установить', 'розыск'], text):
        statuses.append('Отказ, невозможно установить местонахождение должника')
    if has_all_words(['вынести', 'постановление'], text):
        statuses.append('Постановление вынесено')
    if has_all_words(['направить', 'запрос'], text):
        statuses.append('Запрос направлен')
    
    return statuses if statuses else None

In [ ]:
df['answer'] = df['lemmatized_text'].apply(extract_status)
print(df['answer'].value_counts())
print("Всего обработано строк:", df['answer'].value_counts().sum())

answer
[Отказ]                                                                                     130
[Удовлетворено, Частично удовлетворено, Отказ]                                               66
[Запрос направлен]                                                                           66
[Отказ, невозможно установить местонахождение должника]                                      41
[Взыскание обращено]                                                                         35
                                                                                           ... 
[Отказ, Отказ, невозможно установить местонахождение должника, Запрос направлен]              1
[Отказ, Запрет действий, Постановление вынесено, Запрос направлен]                            1
[Отказ, Взыскание обращено, Запрет действий, Постановление вынесено]                          1
[Удовлетворено, Частично удовлетворено, Отказ, Постановление вынесено, Запрос направлен]      1
[Частично удовлетворено, Отказ]  

### Итоги

То есть вышло 62 классов через множество соединённых классов, есть множество классов с <5 примеров, что в итоге не вариант на этом датасете

In [ ]:
# df.to_csv('rule_multilabel_classified.csv', index=False)